## Remover o ultimo campo do label, estão vindo do CVAT 16 campos, mas o formato correto é somente 15

In [ ]:
import glob
import pandas as pd
from IPython.display import display, clear_output

In [ ]:
list_folders = ['pintura3']
#path = '/Users/flavio/Google Drive/Nao compartilhadas/Jobs/getter/codigos/BRF/videos_camera_fixa_segundo_posicionamento/'
path = '/Users/flavio/Downloads/rotulos_madesa_pintura/'

labels_folder = '/labels/'

for folder in list_folders:
    files = glob.glob(path + folder + labels_folder + '*.txt')

    for file in files:
        label = pd.read_csv(file, sep= ' ', header=None)
        
        if label.shape[1] == 16:
            label = label.drop(columns=[15])
            label.to_csv(file,header=None, index=None, sep=' ')

## Pegando total de objetos de cada classe

In [ ]:
label = pd.read_csv(files[0], sep= ' ', header=None)
for file in files:
    label2 = pd.read_csv(file, sep= ' ', header=None)
    label = pd.concat([label, label2])

In [ ]:
label[0].value_counts()

## Criar txt vazio para imagens que não tem rotulo

In [ ]:
import os
import glob

list_folders = ['linha_dupla_combinados']
#path = '/Users/flavio/Google Drive/Nao compartilhadas/Jobs/getter/codigos/BRF/videos_camera_fixa_segundo_posicionamento/'
path = '/Users/flavio/Downloads/rotulos_madesa_pintura/'

labels_folder = '/labels/'
image_folder = '/images/'


In [ ]:
for folder in list_folders:
    files = glob.glob(path + folder + image_folder + '*.PNG')#PNG
    for image_name in files:
        name = image_name.split('/')[-1].split('.PNG')[0]#PNG
        label_name = path + folder + labels_folder + name + '.txt'
        if os.path.isfile(label_name):
            pass
        else:
            #Criar arquivo de label vazio
            with open(label_name, "w") as file:
                pass

## Verificar se BBOXs estão corretos

In [ ]:
import cv2
import glob
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imsave
import numpy as np
%matplotlib inline
from IPython.display import display, clear_output

def desenhar_bounding_box(image, bounding_box):
    # Converter as coordenadas para inteiros
    x_min = int(bounding_box[0])
    y_min = int(bounding_box[1])
    x_max = int(bounding_box[2])
    y_max = int(bounding_box[3])

    # Desenhar o retângulo delimitador na imagem
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return image


In [ ]:
path = '/Users/flavio/Downloads/rotulos_madesa_pintura/pintura2/'
folder_image = '/images/'
folder_label = '/labels/'

files = glob.glob(path + folder_label + '*.txt')

cont=0
for label_file in files:
    labels = pd.read_csv(label_file, sep= ' ', header=None)
    im = plt.imread(path + folder_image + label_file.split('/')[-1].split('.txt')[0] + '.PNG')
    
    for index, label in labels.iterrows():
        im = desenhar_bounding_box(im, list(label[4:8]))
    
    plt.imshow(im)
    plt.show()
    if cont==15:
        break
    cont+=1
    

## Rotacionar BBOX

In [ ]:
import cv2
import glob
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imsave
import numpy as np
%matplotlib inline
from IPython.display import display, clear_output

def desenhar_bounding_box(image, bounding_box):
    # Converter as coordenadas para inteiros
    x_min = int(bounding_box[0])
    y_min = int(bounding_box[1])
    x_max = int(bounding_box[2])
    y_max = int(bounding_box[3])

    # Desenhar o retângulo delimitador na imagem
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    return image

def rotacionar_coordenadas_bbox(image, bounding_box):
    # Obter as dimensões da imagem
    altura, largura, _ = image.shape
    
    x_min, y_min, x_max, y_max = int(bounding_box[0]), int(bounding_box[1]), int(bounding_box[2]), int(bounding_box[3])

    # Rotacionar as coordenadas 90 graus
    x_min_rot = largura - y_max
    y_min_rot = x_min #altura - x_max
    x_max_rot = largura - y_min
    y_max_rot = x_max#altura - x_min
    
    return [x_min_rot, y_min_rot, x_max_rot, y_max_rot]


In [ ]:
path = '/Users/flavio/Downloads/rotulos_madesa_pintura/teste/'
folder_image = '/images/'
folder_label = '/labels/'

files = glob.glob(path + folder_label + '*.txt')

cont=0
for label_file in files:
    labels = pd.read_csv(label_file, sep= ' ', header=None)
    im = plt.imread(path + folder_image + label_file.split('/')[-1].split('.txt')[0] + '.PNG')
    #im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
    
    #Salvando imagem rotacionada
    #imsave(path + folder_image + label_file.split('/')[-1].split('.txt')[0] + '.png', np.uint8(255*im))
    
    for index, label in labels.iterrows():
        new_coords = rotacionar_coordenadas_bbox(im,list(label[4:8]))
        labels.loc[index,4] = new_coords[0]
        labels.loc[index,5] = new_coords[1]
        labels.loc[index,6] = new_coords[2]
        labels.loc[index,7] = new_coords[3]
        #if len(label.shape) == 1:
        #    label.loc[4] = new_coords[0]
        #    label.loc[5] = new_coords[1]
        #    label.loc[6] = new_coords[2]
        #    label.loc[7] = new_coords[3]
        #else:
        #    label.loc[index][4] = new_coords[0]
        #    label.loc[index][5] = new_coords[1]
        #    label.loc[index][6] = new_coords[2]
        #    label.loc[index][7] = new_coords[3]
        
        #im = desenhar_bounding_box(im, rotacionar_coordenadas_bbox(im,list(label[4:8])))
        #im = desenhar_bounding_box(im, list(label[4:8]))
    labels.to_csv(label_file,header=None, index=None, sep=' ')
    
    #Imprimir na mesma linha
    cont+=1
    clear_output(wait=True)
    print(str(cont), '/', len(files))
    
    
    #plt.imshow(im)
    #plt.show()
    #if cont==15:
    #    break
    #cont+=1
    

## Remover imagens que não rotacionaram pq não tem labels

In [ ]:
import glob
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

list_folders = ['video3']#, 'video5', 'video6', 'video7', 'video8', 'video9'] #, 'video10', 'video11', 'video12']
path = '/Users/flavio/Downloads/Rotacionar_kitti_format/'
image_folder = '/image_2/'

for folder in list_folders:
    files = glob.glob(path + folder + image_folder + '*.PNG')
    cont=0
    for im_name in files:
        im = plt.imread(im_name)
        
        if im.shape[0] == 352:
            os.remove(im_name)
        
        cont+=1
        clear_output(wait=True)
        # Imprimir na mesma linha
        print(str(cont), '/', len(files))

## Redimensionar as imagens para 640x640 e as coordenadas dos labels mantendo aspect ratio

## Redimensionar as imagens para 640x640 e as coordenadas dos labels mantendo aspect ratio

In [9]:
import cv2
import pandas as pd
import numpy as np
import glob
from IPython.display import display, clear_output

In [7]:
def resize_coordenadas_bbox(original_size, new_size, bounding_box):
    # Obter as dimensões da imagem
    altura, largura, _ = original_size
    x_min, y_min, x_max, y_max = int(bounding_box[0]), int(bounding_box[1]), int(bounding_box[2]), int(bounding_box[3])
    
    x_scale = new_size[0] / largura
    y_scale = new_size[1] / altura
    # Redimensionar as coordenadas
    x_min_resize = int(np.round(x_min * x_scale))
    y_min_resize = int(np.round(y_min * y_scale))
    x_max_resize = int(np.round(x_max * x_scale))
    y_max_resize = int(np.round(y_max * y_scale))
    
    return [x_min_resize, y_min_resize, x_max_resize, y_max_resize]

def make_square_image_keeping_aspect_ratio(im, labels, original_size):
    #Pegando maior dimensao
    larger_size = original_size[0] if original_size[0] > original_size[1] else original_size[1]
    #Definindo novo tamanho
    new_size = (larger_size, larger_size, original_size[2])
    #Criando imagem toda preta com novo tamanho
    new_image = np.zeros(new_size, dtype=np.uint8)
    
    #Pegando x e y inicial na nova imagem - no cv as coords sao invertidas
    Yt = int((new_size[0] - original_size[0])/2)
    Xt = int((new_size[1] - original_size[1])/2)
    #Criando nova imagem com imagem original centralizada
    new_image[Yt:Yt+original_size[0], Xt:Yt+original_size[1], :] = im
    
    #Alterando o novo labels com BBOXs correto
    for index, label in labels.iterrows():
        labels.loc[index,4] = labels.loc[index,4] + Xt
        labels.loc[index,5] = labels.loc[index,5] + Yt
        labels.loc[index,6] = labels.loc[index,6] + Xt
        labels.loc[index,7] = labels.loc[index,7] + Yt
    
    return new_image, labels

In [8]:
path = '/home/getter-lab/Vídeos/video_1/'

folder_image = '/images/'
folder_label = '/labels/'
new_size = (324, 324)
ext_ori = 'png'
ext_save = 'png'

files = glob.glob(path + folder_label + '*.txt')
cont=0
for label_file in files:
    labels = pd.read_csv(label_file, sep= ' ', header=None)
    im = cv2.imread(path + folder_image + label_file.split('/')[-1].split('.txt')[0] + '.' + ext_ori)
    original_size = im.shape
    #Deixando imagem quadrada mantendo aspect ratio
    new_image_square, labels = make_square_image_keeping_aspect_ratio(im, labels, original_size)
        
    #Redimensionamento
    im = cv2.resize(new_image_square, new_size)
    
    #Salvando imagem nova_dimensao
    cv2.imwrite(path + folder_image + label_file.split('/')[-1].split('.txt')[0] + '.' + ext_save, im)
    
    for index, label in labels.iterrows():
        new_coords = resize_coordenadas_bbox(new_image_square.shape, new_size, list(label[4:8]))
        labels.loc[index,4] = new_coords[0]
        labels.loc[index,5] = new_coords[1]
        labels.loc[index,6] = new_coords[2]
        labels.loc[index,7] = new_coords[3]
    labels.to_csv(label_file, header=None, index=None, sep=' ')
    
    #Imprimir na mesma linha
    cont+=1
    clear_output(wait=True)
    print(str(cont), '/', len(files))

506 / 506
